# setup

In [65]:
import torch
import yaml
import random
import numpy as np
import sys

from tqdm import tqdm
from torch.utils.data.dataloader import DataLoader

sys.path.append('../utils')
from dataset import ECG_TEXT_Dsataset
from builder import ECGCLIP
from utils import find_best_thresholds, metrics_table
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
config = yaml.load(open("config.yaml", "r"), Loader=yaml.FullLoader)
model_checkpoints_folder = '../checkpoints/'

In [5]:
torch.manual_seed(42)
random.seed(0)
np.random.seed(0)

# dataset

In [6]:
# data_path = config['dataset']['data_path']
data_path = '\\Users\katri\Downloads\git\lesaude\code\CODEmel'
dataset = ECG_TEXT_Dsataset(
    data_path=data_path, dataset_name=config['dataset']['dataset_name'])
# train_dataset = dataset.get_dataset(train_test='train')
val_dataset = dataset.get_dataset(train_test='val')
tst_dataset = dataset.get_dataset(train_test = 'test')
batch_size = 4
val_loader = DataLoader(val_dataset, batch_size = batch_size, shuffle=False, )
tst_loader = DataLoader(tst_dataset, batch_size = batch_size, shuffle=False, )

Load CODEmel dataset!
train size: 35995
val size: 2006
tst size: 1999
total size: 40000
Apply Val-stage Transform!
val dataset length:  2006
Apply Val-stage Transform!
test dataset length:  1999


# builder

In [7]:
model = ECGCLIP(config['network'])

In [8]:
model = model.to(device).eval()
ckpt = torch.load(model_checkpoints_folder + config['wandb_name'] + f'_bestZeroShotAll_ckpt.pth', map_location='cpu')
model.load_state_dict(ckpt)

C:\Users\katri\AppData\Local\Temp\ipykernel_14940\2575387603.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(model_checkpoints_folder + config['wandb_n

<All keys matched successfully>

# zeroshot

In [ ]:
# # text_normal = 'ritmo sinusal regular. sem desvio de eixo. onda p: amplitude e duracao normais. pri: duracao normal. qrs: duracao, eixo, morfologia e amplitude normais. st: sem supra ou infradesnivelamento. onda t: morfologia habitual. qtc: duracao normal. conclusao: 1- ecg dentro dos limites da normalidade. o tracado impresso corresponde a apenas um trecho do registro eletrocardiografico. este laudo foi elaborado utilizando-se todo o tracado disponivel no sistema.'
# text_1davb = 'ritmo sinusal regular. sem desvio de eixo. onda p: amplitude e duracao normais. pri: duracao aumentadata >200 ms. qrs: duracao, eixo, morfologia e amplitude normais. st: sem supra ou infradesnivelamento. onda t: morfologia habitual. qtc: duracao normal. conclusao: 1- bloqueio atrioventricular de primeiro grau. o tracado impresso corresponde a apenas um trecho do registro eletrocardiografico. este laudo foi elaborado utilizando-se todo o tracado disponivel no sistema.'
# text_rbbb = 'ritmo sinusal regular. sem desvio de eixo. onda p: amplitude e duracao normais. pri: duracao normal. qrs: eixo e amplitudes normais. duracao aumentada. morfologia brd. st e onda t: alteracoes secundarias ao brd. qtc: duracao normal. conclusao: 1- bloqueio de ramo direito. o tracado impresso corresponde a apenas um trecho do registro eletrocardiografico. este laudo foi elaborado utilizando-se todo o tracado disponivel no sistema.'
# text_lbbb = 'ritmo sinusal regular. sem desvio de eixo. onda p: amplitude e duracao normais. pri: duracao normal. qrs: eixo e amplitudes normais. duracao aumentada. morfologia bre. st e onda t: alteracoes secundarias ao bre. qtc: duracao normal. conclusao: 1- bloqueio de ramo esquerdo. o tracado impresso corresponde a apenas um trecho do registro eletrocardiografico. este laudo foi elaborado utilizando-se todo o tracado disponivel no sistema.'
# text_sb = 'ritmo sinusal regular com frequencia fc de bpm. sem desvio de eixo. onda p: amplitude e duracao normais. pri: duracao normal. qrs: duracao, eixo, morfologia e amplitude normais. st: sem supra ou infradesnivelamento. onda t: morfologia habitual. qtc: ms duracao normal. conclusao: 1- bradicardia sinusal (fc bpm). o tracado impresso corresponde a apenas um trecho do registro eletrocardiografico. este laudo foi elaborado utilizando-se todo o tracado disponivel no sistema.'
# text_af = 'ritmo irregular. sem desvio de eixo. ausencia de onda p. pri: duracao normal. qrs: duracao, eixo, morfologia e amplitude normais. st: sem supra ou infradesnivelamento. onda t: morfologia habitual. qtc: duracao normal. conclusao: 1- fibrilacao atrial. o tracado impresso corresponde a apenas um trecho do registro eletrocardiografico. este laudo foi elaborado utilizando-se todo o tracado disponivel no sistema.'
# text_st = 'ritmo sinusal regular com frequencia fc de bpm. sem desvio de eixo. onda p: amplitude e duracao normais. pri: duracao normal. qrs: duracao, eixo, morfologia e amplitude normais. st: sem supra ou infradesnivelamento. onda t: morfologia habitual. qtc: ms duracao normal. conclusao: 1- taquicardia sinusal (fc bpm). o tracado impresso corresponde a apenas um trecho do registro eletrocardiografico. este laudo foi elaborado utilizando-se todo o tracado disponivel no sistema.'

# texts = [text_1davb, text_rbbb, text_lbbb, text_sb, text_af, text_st]
# num_classes = len(texts)

In [67]:
ritmoondap_normal = 'ritmo sinusal regular. onda p: amplitude e duracao normais.'
ritmoondap_af = 'ritmo irregular. ausencia de onda p.'

pri_normal = 'pri: duracao normal.'
pri_1davb = 'pri: duracao aumentadata >200 ms.'

qrsstondat_normal = 'qrs: duracao, eixo, morfologia e amplitude normais. st: sem supra ou infradesnivelamento. onda t: morfologia habitual.'
qrsstondat_rbbb = 'qrs: eixo e amplitudes normais. duracao aumentada. morfologia brd. st e onda t: alteracoes secundarias ao brd.'
qrsstondat_lbbb = 'qrs: eixo e amplitudes normais. duracao aumentada. morfologia bre. st e onda t: alteracoes secundarias ao bre.'

# texts = [ritmoondap_normal, ritmoondap_af]
# texts = [ritmo_normal, ritmo_af]
# texts = [ondap_normal, ondap_af]
# texts = [pri_normal, pri_1davb]
# texts = [qrs_normal, qrs_rbbb, qrs_lbbb]
# texts = [stondat_normal, stondat_rbbb, stondat_lbbb]
texts = [qrsstondat_normal, qrsstondat_rbbb, qrsstondat_lbbb]
num_classes = len(texts)

In [68]:
with torch.no_grad():
    # text synthesis
    zeroshot_weights = []
    for text in tqdm(texts):
        text = model._tokenize([text.lower()])

        class_embeddings = model.get_text_emb(text.input_ids.to(device=device), text.attention_mask.to(device=device)) # embed with text encoder
        class_embeddings = model.proj_t(class_embeddings) # embed with text encoder

        # normalize class_embeddings
        class_embeddings /= class_embeddings.norm(dim=-1, keepdim=True)
        # average over templates 
        class_embedding = class_embeddings.mean(dim=0) 
        # norm over new averaged templates
        class_embedding /= class_embedding.norm()

        zeroshot_weights.append(class_embedding)
    zeroshot_weights = torch.stack(zeroshot_weights, dim=1)

100%|██████████| 3/3 [00:02<00:00,  1.20it/s]


In [69]:
y_pred = []
all_binary_results = []
all_true_labels = []
with torch.no_grad():
    for data in tqdm(val_loader):
        # read
        # report = data['raw_text']
        ecg = data['ecg'].to(torch.float32).to(device).contiguous()
        label = data['label'].float().to(device)
        exam_id = data['exam_id'].to(device)

        # predict
        ecg_emb = model.ext_ecg_emb(ecg)
        ecg_emb /= ecg_emb.norm(dim=-1, keepdim=True)

        # obtain logits (cos similarity)
        logits = ecg_emb @ zeroshot_weights
        logits = torch.squeeze(logits, 0) # (N, num_classes)
        # logits = torch.stack([torch.max(logits[:, s], axis = 1).values for s in slices]).T

        # norm_logits = (logits - logits.mean()) / (logits.std())
        norm_logits = (logits - logits.mean(axis = 1).unsqueeze(1)) / (logits.std(axis = 1).unsqueeze(1))
        probs = torch.sigmoid(norm_logits)
        # probs = torch.sigmoid(logits)
        # break

        binary_result = torch.zeros_like(probs)
        binary_result[np.arange(logits.shape[0]), torch.argmax(probs, axis = 1)] = 1

        y_pred.append(probs)
        all_binary_results.append(binary_result)
        all_true_labels.append(label)

100%|██████████| 502/502 [01:01<00:00,  8.19it/s]


In [70]:
y_pred = torch.cat(y_pred, dim=0)
all_binary_results = torch.cat(all_binary_results, dim=0)
all_true_labels = torch.cat(all_true_labels, dim=0)

In [ ]:
# probs, data['raw_text'], label

In [71]:
all_binary_results

tensor([[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        ...,
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])

In [72]:
all_true_labels

tensor([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])

In [74]:
# f1_score(all_true_labels[:, 4], all_binary_results[:, 1], zero_division=0) # af
# f1_score(all_true_labels[:, 1], all_binary_results[:, 1], zero_division=0) # rbbb
f1_score(all_true_labels[:, 2], all_binary_results[:, 2], zero_division=0) # lbbb

np.float64(0.32222222222222224)

In [39]:
with torch.no_grad():
    for data in tqdm(val_loader):
            # read
            report = data['raw_text']
            # ecg = data['ecg'].to(torch.float32).to(device).contiguous()
            label = data['label'].float().to(device)
            exam_id = data['exam_id'].to(device)

            # predict
            text = model._tokenize(report)
            class_embeddings = model.get_text_emb(text.input_ids.to(device=device), text.attention_mask.to(device=device)) # embed with text encoder
            class_embeddings = model.proj_t(class_embeddings) # embed with text encoder

            # normalize class_embeddings
            class_embeddings /= class_embeddings.norm(dim=-1, keepdim=True)
            # average over templates 
            class_embedding = class_embeddings
            # norm over new averaged templates
            class_embedding /= class_embedding.norm()

            # obtain logits (cos similarity)
            logits = class_embedding @ zeroshot_weights
            logits = torch.squeeze(logits, 0) # (N, num_classes)
            # logits = torch.stack([torch.max(logits[:, s], axis = 1).values for s in slices]).T

            # norm_logits = (logits - logits.mean()) / (logits.std())
            norm_logits = (logits - logits.mean(axis = 1).unsqueeze(1)) / (logits.std(axis = 1).unsqueeze(1))
            probs = torch.sigmoid(norm_logits)
            break

  0%|          | 0/502 [00:02<?, ?it/s]


In [42]:
probs, data['raw_text'], label

(tensor([[0.2488, 0.6991, 0.5650],
         [0.7601, 0.3709, 0.3486],
         [0.7582, 0.3948, 0.3284],
         [0.5568, 0.7040, 0.2508]]),
 ['Ritmo atrial ectópico. Extrassístoles supraventriculares isoladas. Extrassístoles ventriculares isoladas. Sem desvio de eixo elétrico. Onda P: baixa amplitude e morfologia de ritmo atrial ectópico em DII, DIII e aVF. PRi: duração normal. QRS: duração e morfologia normais. Baixa amplitude do complexo QRS de DI a aVF. ST: sem supra ou infradesnivelamento. Onda T: morfologia habitual. QTc: duração normal. Conclusão: 1- Ritmo atrial ectópico. 2- Extrassístoles supraventriculares isoladas. 3- Extrassístoles ventriculares isoladas. 4- Baixa voltagem do complexo QRS no plano frontal (efeito dielétrico). Dr. Otaviano da Silva Júnior',
  'RITMO SINUSAL REGULAR COM FREQUENCIA CARDIACA NORMAL. SEM DESVIO DE EIXO ELETRICO. ONDA P: AMPLITUDE E DURACAO NORMAIS. PRI: DURACAO NORMAL. QRS: DURACAO, MORFOLOGIA E AMPLITUDE NORMAIS. ST: SEM SUPRA OU INFRADESNIVEL

In [ ]:
with torch.no_grad():
    # text synthesis
    zeroshot_weights = []
    for text in tqdm(texts):
        text = model._tokenize([text.lower()])

        class_embeddings = model.get_text_emb(text.input_ids.to(device=device), text.attention_mask.to(device=device)) # embed with text encoder
        class_embeddings = model.proj_t(class_embeddings) # embed with text encoder

        # normalize class_embeddings
        class_embeddings /= class_embeddings.norm(dim=-1, keepdim=True)
        # average over templates 
        class_embedding = class_embeddings.mean(dim=0) 
        # norm over new averaged templates
        class_embedding /= class_embedding.norm()

        zeroshot_weights.append(class_embedding)
    zeroshot_weights = torch.stack(zeroshot_weights, dim=1)

    # val thresholds
    thresholds = np.arange(0, 1.01, 0.01)
    predictions = {thresh: [[] for _ in range(num_classes)] for thresh in thresholds}
    true_labels_dict = [[] for _ in range(num_classes)]
    for data in tqdm(val_loader):
        # read
        # report = data['raw_text']
        ecg = data['ecg'].to(torch.float32).to(device).contiguous()
        label = data['label'].float().to(device)
        exam_id = data['exam_id'].to(device)

        # predict
        ecg_emb = model.ext_ecg_emb(ecg)
        ecg_emb /= ecg_emb.norm(dim=-1, keepdim=True)

        # obtain logits (cos similarity)
        logits = ecg_emb @ zeroshot_weights
        logits = torch.squeeze(logits, 0) # (N, num_classes)
        logits = torch.stack([torch.max(logits[:, s], axis = 1).values for s in slices]).T

        # norm_logits = (logits - logits.mean()) / (logits.std())
        norm_logits = (logits - logits.mean(axis = 1).unsqueeze(1)) / (logits.std(axis = 1).unsqueeze(1))
        probs = torch.sigmoid(norm_logits)

        for class_idx in range(num_classes):
            for thresh in thresholds:
                predicted_binary = (probs[:, class_idx] >= thresh).float()
                predictions[thresh][class_idx].extend(predicted_binary.cpu().numpy())
            true_labels_dict[class_idx].extend(label[:, class_idx].cpu().numpy())
    best_f1s, best_thresholds = find_best_thresholds(predictions, true_labels_dict, thresholds)

    # test
    y_pred = []
    all_binary_results = []
    all_true_labels = []
    for data in tqdm(tst_loader):
        # read
        # report = data['raw_text']
        ecg = data['ecg'].to(torch.float32).to(device).contiguous()
        label = data['label'].float().to(device)
        exam_id = data['exam_id'].to(device)

        # predict
        ecg_emb = model.ext_ecg_emb(ecg)
        ecg_emb /= ecg_emb.norm(dim=-1, keepdim=True)

        # obtain logits (cos similarity)
        logits = ecg_emb @ zeroshot_weights
        logits = torch.squeeze(logits, 0) # (N, num_classes)
        logits = torch.stack([torch.max(logits[:, s], axis = 1).values for s in slices]).T

        # norm_logits = (logits - logits.mean()) / (logits.std())
        norm_logits = (logits - logits.mean(axis = 1).unsqueeze(1)) / (logits.std(axis = 1).unsqueeze(1))
        probs = torch.sigmoid(norm_logits)

        binary_result = torch.zeros_like(probs)
        for i in range(len(best_thresholds)):
            binary_result[:, i] = (probs[:, i] >= best_thresholds[i]).float()

        y_pred.append(logits)
        all_binary_results.append(binary_result)
        all_true_labels.append(label)
        
    y_pred = torch.cat(y_pred, dim=0)
    all_binary_results = torch.cat(all_binary_results, dim=0)
    all_true_labels = torch.cat(all_true_labels, dim=0)